Import dependencies and libraries


In [2]:
import boto3
import json
import logging
import requests
import pandas as pd
import csv


In [30]:
import boto3
import json
import logging
import requests
import pandas as pd


dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('itemstore-Items')

with open('items3.json') as json_file:
    items = json.load(json_file)
    with open("./item2.csv", "w") as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            columnheader=[]
            columnheader.append("ITEM_ID")
            columnheader.append("GENRE")
            writer.writerow(columnheader)
            for x in items:
                asin = x['asin']
                if asin == "empty":
                    continue
                    
                title = x['title']
                if title == "empty":
                    continue
                    
                imUrl = x['imUrl']

                if imUrl == "empty":
                    continue
                
                try:
                    img = requests.get(imUrl).content
                except InvalidImageFormatException:
                    continue

                client = boto3.client('rekognition')
                response = client.detect_labels(
                Image={
                    "Bytes" : img
                },
                MaxLabels=123,
                MinConfidence=60
                )

                print ("************")
                for key, value in response.items():
                    if key == 'Labels':
                        labels = value

                df = pd.DataFrame(labels)
                if df.empty:
                    continue
                df2 = df

                for col in df2.columns: 
                    if col == "Parents":
                        df2=df2.drop(['Parents'], axis=1)
                    elif col == "Instances":
                        df2=df2.drop(['Instances'], axis=1)
                print ("******")
                print (df2)
                jsonOb = df2["Name"].to_json(orient='records')

                table.put_item(
                    Item={
                        'asin': asin,
                        'title': title,
                        'imUrl':imUrl, 
                        'rekognition': jsonOb

                    }
                )
                
                csvrow = []
                categories =""
                for x in df2["Name"]:
                    categories += x
                    categories += "|"
    
                csvrow.append(asin)
                csvrow.append (categories)
                writer.writerow(csvrow)


************
******
   Confidence        Name
0   99.831238       Drink
1   99.831238        Soda
2   99.831238    Beverage
3   94.991508        Food
4   94.991508     Ketchup
5   76.706963  Pop Bottle
6   76.706963      Bottle
7   70.760788        Coca
8   70.760788        Coke
************
******
   Confidence         Name
0   90.163925       Wallet
1   90.163925  Accessories
2   90.163925    Accessory
3   73.416534      Adapter
************
******
   Confidence          Name
0   97.263611        Camera
1   97.263611   Electronics
2   82.886307  Video Camera
3   60.485237    Binoculars
************
******
   Confidence           Name
0   91.494095  Digital Watch
************
******
   Confidence       Name
0   95.222954  Lamp Post
1   83.826889       Oars
2   61.786564     Symbol
************
******
   Confidence      Name
0   80.437836      Rope
1   67.307632  Clothing
2   67.307632   Apparel
************
******
   Confidence        Name
0   88.868462      Sports
1   88.868462      

************
******
   Confidence        Name
0   97.690552      Weapon
1   97.690552    Weaponry
2   93.401321  Ammunition
3   78.115356      Bullet
************
******
   Confidence        Name
0   97.562164    Weaponry
1   97.562164      Weapon
2   94.662827  Ammunition
3   81.230980      Bullet
************
******
   Confidence        Name
0   92.194183         Rug
1   89.090820  Bath Towel
2   89.090820       Towel
************
******
   Confidence    Name
0   79.045395    Tool
1   77.372643   Clamp
2   63.103390  Buckle
************
******
   Confidence   Name
0   99.695999   Tool
1   97.738121  Clamp
************
******
   Confidence         Name
0    99.96769  Accessories
1    99.96769    Accessory
2    99.96769   Sunglasses
3    99.46656      Glasses
4    83.92128      Goggles
************
******
   Confidence      Name
0   92.597496  Lighting
1   66.391853       Bow
2   63.401180   Machine
************
******
   Confidence           Name
0   94.830536           Tool
1   85.31

************
******
   Confidence      Name
0   88.782188      Tool
1   64.387497  Weaponry
2   64.387497    Weapon
3   64.387497     Blade
4   62.808331   Handsaw
5   62.808331   Hacksaw
6   60.588249     Knife
************
******
   Confidence          Name
0   99.984337      Clothing
1   99.984337       Apparel
2   99.215866           Cap
3   99.215866  Baseball Cap
4   99.215866           Hat
5   67.547684       Sun Hat
************
******
   Confidence  Name
0   85.833984  Rope
************
******
   Confidence      Name
0   99.937042    Weapon
1   99.937042       Gun
2   99.937042  Weaponry
3   98.770027   Handgun
************
******
   Confidence  Name
0   85.709328  Text
************
******
    Confidence          Name
0    93.992134           Cap
1    93.992134      Clothing
2    93.992134  Baseball Cap
3    93.992134       Apparel
4    93.992134           Hat
5    88.027199        Fossil
6    78.663887     Accessory
7    78.663887   Accessories
8    76.515747      Gemstone
9 

************
******
   Confidence      Name
0   98.710693  Clothing
1   98.710693   Apparel
2   93.051170     Human
3   93.051170    Person
4   77.359718    People
5   66.849113     Shirt
************
******
   Confidence        Name
0   99.987587    Clothing
1   99.987587     Apparel
2   86.668030       Human
3   86.668030      Person
4   86.124847       Shirt
5   82.609993     T-Shirt
6   70.677666  Undershirt
7   62.109196      Sleeve
************
******
   Confidence         Name
0   98.743423     Exercise
1   98.743423  Working Out
2   98.743423        Human
3   98.743423        Sport
4   98.743423       Person
5   98.743423       Sports
6   98.568970      Fitness
7   96.008553          Gym
8   87.774765      Sitting
************
******
   Confidence        Name
0   83.965256       Plant
1   69.801559    Outdoors
2   60.906281  Vegetation
************
******
   Confidence         Name
0   98.578407       Person
1   98.578407        Human
2   98.192810  Working Out
3   98.192810   

************
******
   Confidence      Name
0   96.735382     Blade
1   96.735382    Weapon
2   96.735382     Knife
3   96.735382  Weaponry
4   74.745865  Scissors
************
******
   Confidence       Name
0   99.596222  First Aid
1   98.309662       Text
2   98.309662      Label
3   97.589661    Bandage
************
******
    Confidence        Name
0    98.860443        Word
1    90.919319        Text
2    89.422928    Alphabet
3    86.601387      Number
4    86.601387      Symbol
5    75.774323  Volleyball
6    75.774323  Team Sport
7    75.774323      Sports
8    75.774323       Sport
9    75.774323        Team
10   75.239571    Clothing
11   75.239571     Apparel
12   61.651840  Scoreboard
************
******
   Confidence           Name
0   85.645134  Digital Watch
1   77.711876           Pump
2   77.711876       Gas Pump
3   77.711876        Machine
4   74.658165     Wristwatch
************
************
******
   Confidence       Name
0   99.686768       Tire
1   82.748657   

...

************
******
   Confidence      Name
0   98.617188     Human
1   98.617188    Person
2   92.353546     Blade
3   92.353546     Razor
4   92.353546  Weaponry
5   92.353546    Weapon
6   89.464035     Knife
************
******
   Confidence               Name
0   85.333122  Electrical Device
1   75.863174              Wedge
2   75.051926         Microphone
************
******
   Confidence         Name
0   99.714798         Tool
1   99.714798          Axe
2   96.402130  Electronics
3   86.101181       Hammer
4   61.616360         Lamp
************
******
   Confidence          Name
0   96.947517     Appliance
1   96.947517         Mixer
2   89.123947    Coffee Cup
3   89.123947           Cup
4   81.466141         Drink
5   81.466141      Beverage
6   76.718307       Machine
7   73.278450  Space Heater
8   73.278450        Heater
************
******
   Confidence        Name
0   98.901108      Weapon
1   98.901108    Weaponry
2   98.901108       Blade
3   98.901108    Scissors
4   

************
******
   Confidence       Name
0   98.437782  Cosmetics
1   89.061470     Bottle
************
******
   Confidence        Name
0   91.039482       Light
1   84.508873  Flashlight
2   84.508873        Lamp
3   64.806412    Lighting
************
******
    Confidence          Name
0    99.947357   Electronics
1    99.947357    Cell Phone
2    99.947357  Mobile Phone
3    99.947357         Phone
4    84.964554   Soccer Ball
5    84.964554          Ball
6    84.964554    Team Sport
7    84.964554        Sports
8    84.964554         Sport
9    84.964554          Team
10   84.964554        Soccer
11   84.964554        People
12   84.964554         Human
13   84.964554      Football
14   84.964554        Person
15   61.994987       Apparel
16   61.994987      Clothing
************
******
   Confidence           Name
0   85.053635     Flashlight
1   85.053635           Lamp
2   78.960403  Advertisement
3   78.960403          Flyer
4   78.960403       Brochure
5   78.960403      

************
******
   Confidence  Name
0   76.669609  Text
1   63.465012  Wood
************
******
   Confidence          Name
0   99.999657      Clothing
1   99.999657       Apparel
2   98.505539      Swimwear
3   97.275513           Hat
4   97.275513           Cap
5   97.275513  Swimming Cap
6   92.307793   Bathing Cap
7   81.781624          Tape
************
******
    Confidence          Name
0    97.479210        Sports
1    97.479210         Sport
2    96.924080          Team
3    96.924080    Team Sport
4    93.848877    Basketball
5    92.881149        Helmet
6    92.881149       Apparel
7    92.881149      Clothing
8    84.983383           Cap
9    84.983383           Hat
10   84.983383  Baseball Cap
************
******
   Confidence         Name
0   82.079720      Cushion
1   76.029686  Accessories
2   76.029686    Accessory
3   68.814095        Heart
4   66.899261       Pillow
5   63.040398     Wax Seal
************
******
   Confidence     Name
0   72.934219  Machine
1   6

************
******
   Confidence Name
0   60.331802  Pen
************
******
   Confidence        Name
0   97.708458       Linen
1   97.708458  Home Decor
2   67.550926       Paper
************
******
   Confidence       Name
0   98.811630     Shaker
1   98.811630     Bottle
2   80.777115  Appliance
3   80.777115     Cooker
************
******
   Confidence    Name
0   99.951950  Tripod
1   98.257416     Bow
************
******
   Confidence       Name
0   96.290672  Appliance
1   94.179817       Oven
2   91.125580      Mixer
3   76.432663      Stove
4   68.828644     Cooker
************
******
   Confidence         Name
0   87.893684  Electronics
************
******
   Confidence Name
0   88.983246  Mat
************
******
   Confidence     Name
0   64.967941  Machine
************
******
   Confidence        Name
0   90.266373    Weaponry
1   90.266373      Weapon
2   78.857033         Gun
3   61.967201       Strap
4   61.449287  Ammunition
************
******
   Confidence         N

************
******
   Confidence         Name
0   99.998360  Accessories
1   99.998360   Sunglasses
2   99.998360    Accessory
3   89.727524      Goggles
4   82.776161        Glass
************
******
   Confidence      Name
0   87.174698      Bomb
1   87.174698    Weapon
2   87.174698  Weaponry
3   61.209637  Dynamite
************
******
   Confidence    Name
0   90.369530    Tool
1   83.215187     Axe
2   75.687004  Wrench
************
******
   Confidence          Name
0   98.148590   Electronics
1   98.148590        Camera
2   83.992317  Video Camera
3   62.448296        Webcam
************
******
   Confidence      Name
0   86.289360  Outdoors
1   78.034637    Nature
2   77.108261     Strap
************
******
   Confidence     Name
0   98.749176  Machine
1   92.600281     Gear
2   65.222244     Tool
3   65.222244    Brush
************
******
   Confidence      Name
0   99.994423  Weaponry
1   99.994423    Weapon
2   99.994423       Gun
3   97.830574   Handgun
************
******

In [ ]:
 with open("./item.csv", "w") as csv_file:
            writer = csv.writer(csv_file, delimiter=',')

In [ ]:
csvrow = []
                categories =""
                for x in df2["Categories"]:
                    categories += x
                    categories += "|"
    
                csvrow.append(asin)
                csvrow.append (categories)
                writer.writerow(csvrow)
            